In [1]:
import wlogs as wl

from datetime import datetime, timedelta

In [2]:
FROM_DATE = datetime(year=2023, month=1, day=23)

In [3]:
guild = wl.GUILD_ID
client = wl.graphQL_client_from_json()
reports = wl.load_latest_reports()

In [4]:
def convert_time(epoch_time_ms):
    return datetime.fromtimestamp(float(epoch_time_ms)/1000)

In [5]:
reports['start'] = reports['startTime'].apply(convert_time)
reports['end'] = reports['endTime'].apply(convert_time)
reports['duration'] = reports['end'] - reports['start']

reports = reports[(reports['segments'] > 0) & (reports['duration'] > timedelta(hours=0)) & (reports['duration'] < timedelta(hours=5)) & (reports['start'] >= FROM_DATE)]

In [6]:
reports_codes = reports['code'].tolist()
#TODO : remove codes already requested and save into json file

In [7]:
for code in reports_codes:
    response = client.post(wl.query_report(code))
    wl.save_report_data_to_json(response, code)